In [1]:
import os
import json
import glob
import re
from src.wimbd_ import _load_dataset

HF_HOME = "/share/edc/home/antonis/datasets/huggingface"
save_path = os.makedirs(os.path.join(HF_HOME, 'wmt09_gens'), exist_ok=True)

base_pth = "/share/edc/home/antonis/LLM-Incidental-Supervision/incidental-supervision/models/experiment_6_logits_max_4/inference/allenai/OLMo-7B/TRANSLATION"
doc_res_files = glob.glob(os.path.join(base_pth, "**/doc_results.json"), recursive=True)


[nltk_data] Downloading package punkt to
[nltk_data]     /local/home/antonis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /local/home/antonis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[dynet] random seed: 1234
[dynet] allocating memory: 32MB
[dynet] memory allocation done.


In [2]:
pattern = re.compile(r'wmt\d{2}-(\w{2}-\w{2})')
language_pairs = [pattern.search(path).group(1) for path in doc_res_files if pattern.search(path)]
doc = json.load(open(doc_res_files[0]))

def find_lang(path):
    pattern = re.compile(r'wmt\d{2}-(\w{2}-\w{2})')
    return pattern.search(path).group(1)

In [3]:
doc['wmt09-en-hu'][0]


{'src': 'In New York, Wall Street opened with significant losses.',
 'ref': 'New Yorkban a Wall Street jelentős veszteséggel nyitott.',
 'result': [' A New York-i Wall Street előtt körülbelül 30 perc alatt kezdődik.'],
 'bleu': 8.054496384843702,
 'id': 0}

In [4]:
doc_res_files[0].split('/')[-3]


'wmt09-en-hu'

In [5]:
wmt = _load_dataset('wmt')


Loading dataset wmt
Loading dataset for language pair ('cs', 'en')
Loading dataset for language pair ('de', 'en')
Loading dataset for language pair ('fr', 'en')
Loading dataset for language pair ('es', 'en')
Loading dataset for language pair ('it', 'en')
Loading dataset for language pair ('hu', 'en')
Loading dataset for language pair ('en', 'cs')
Loading dataset for language pair ('en', 'de')
Loading dataset for language pair ('en', 'fr')
Loading dataset for language pair ('en', 'es')
Loading dataset for language pair ('en', 'it')
Loading dataset for language pair ('en', 'hu')


In [8]:
wmt['cs-en']['translation']

from tqdm import tqdm
from datasets import DatasetDict
from datasets import Dataset

def insert_gens_into_wmt(wmt, doc_res_files):
    for path in tqdm(doc_res_files, desc="Processing files"):
        with open(path, 'r') as f:
            doc_res = json.load(f)
        
        task = path.split('/')[-3]
        lang = find_lang(path)
        lang1, lang2 = lang.split('-')
        total_rows = len(wmt[lang]['translation'])
        matched_rows = 0
        doc_res_task_dict = wmt[lang].to_dict()
        len_task_dict = len(doc_res_task_dict['translation'])
        unmatched_rows = [i for i in range(len_task_dict)]

        for doc in tqdm(doc_res[task], desc=f"Inserting generations for {task}", leave=False):
            src = doc['src']
            ref = doc['ref']
            gen = doc['result'][0]

            for idx in unmatched_rows:
                row = doc_res_task_dict['translation'][idx]
                print(row)
                if row[lang1] != src and row[lang2] != ref:
                    continue
                else:
                    doc_res_task_dict['translation'][idx]['gen'] = gen
                    matched_rows += 1
                    unmatched_rows.remove(idx)
                    break
        
        if len(unmatched_rows) > 0:
            print(f"Could not match {len(unmatched_rows)} rows for {task}")

        wmt[lang] = Dataset.from_dict(doc_res_task_dict)
        dataset_dict = {lang: Dataset.from_dict({'translation': value['translation']}) for lang, value in wmt.items()}
        dataset_dict = DatasetDict(dataset_dict)
        # Now you can save it to disk
        dataset_dict.save_to_disk(os.path.join(HF_HOME, 'wmt09_gens'))
        print(f"Matched {matched_rows}/{total_rows} rows for {task}")

    return wmt


wmt = insert_gens_into_wmt(wmt, doc_res_files)

dataset_dict = {lang: Dataset.from_dict({'translation': value['translation']}) for lang, value in wmt.items()}
dataset_dict = DatasetDict(dataset_dict)

# Now you can save it to disk
dataset_dict.save_to_disk(os.path.join(HF_HOME, 'wmt09_gens'))


Processing files:   0%|          | 0/12 [00:00<?, ?it/s]

Processing files:   0%|          | 0/12 [00:00<?, ?it/s]

{'en': 'Prague Stock Market falls to minus by the end of the trading day', 'gen': ' A New York-i Wall Street előtt körülbelül 30 perc alatt kezdődik.', 'hu': 'A prágai tőzsde a kereskedés végére mínuszba került'}


KeyError: 'lang1'